# Setting up Compass, Sass and Django

Brandon Rhodes gives a [great tutorial](http://rhodesmill.org/brandon/2011/adding-compass/) on how to add Compass to a [Django](https://www.djangoproject.com/) project. In that tutorial, the first step is to create a `compass` folder, and inside that folder, run an `install.sh` script to create the gems required for Compass.  I had trouble with the line inside this script that looks like this:

```shell
BASE=$(dirname $(readlink -f $(which "$0")))
```

For some reason on my Mac, `readlink` was missing. Instead, I [found](http://stackoverflow.com/questions/59895/can-a-bash-script-tell-what-directory-its-stored-in) a different way of obtaining the folder in which a script resides, and so my `install.sh` looks like this:

```shell
#!/bin/bash
# compass/install.sh - install Compass under the "./Gem" directory

if ! which gem >/dev/null ;then
    echo 'Error: no "gem" command available'
    echo 'Please install Ruby.'
    exit 1
fi
BASE=$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )
cd $BASE  # the directory where this script lives
gem install -i Gem compass
gem install -i Gem compass-susy-plugin
```

The same occurs in the second script, `compass.sh`:

```shell
#!/bin/bash
# compass/compass.sh - properly invoke the "Compass" program

BASE=$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )
export GEM_HOME=$BASE/Gem
export RUBYLIB=$BASE/Gem/lib
$BASE/Gem/bin/compass "$@"
```

The remainder of the tutorial remains the same.

<h3 style="color: red; ">Edit: 14 Aug 2015</h3>

## There is a better way

There are tools that make this all much easier.  Once you have a ruby installed and working, the first step is to install [bundler](http://bundler.io):

```bash
$ sudo gem install bundler
```

After that, you need to add a `Gemfile` to your Django project.  I put mine alongside my `requirements.txt`, for all the same reasons.  The contents of the Gemfile look something like this:

```ruby
source 'https://rubygems.org'

gem 'sass'
gem 'compass'
gem 'breakpoint'
gem 'singularitygs'
gem 'color-schemer'
```

Note: dependencies can be version-clamped. I haven't done that here, but the information is easily found online.

Then, to install these gems, you simply (in the same folder as the `Gemfile`):

```bash
$ bundle install
```

Thereafter, to run any of the gems on your _Sass_ files, you can execute the bundle version of gem executables like so:

```bash
$ bundle exec compass watch
```

In my case, I've installed the wonderful [livereload](https://github.com/lepture/python-livereload) (with `pip install livereload`) and I've modified the Django `manage.py` file to use livereload for the development server when launched with

```bash
$ python manage.py livereload
```

So, my `manage.py` looks like this:

```python
#!/usr/bin/env python
import os
import sys


if __name__ == "__main__":
    # CHANGED manage.py will use development settings by
    # default. Change the DJANGO_SETTINGS_MODULE environment variable
    # for using the environment specific settings file.
    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproj.settings.development")

    from django.core.management import execute_from_command_line

    if 'livereload' in sys.argv:
        from django.core.wsgi import get_wsgi_application
        from livereload import Server, shell
        application = get_wsgi_application()
        server = Server(application)

        # Add watches - plain files
        folders = ['myproj/', 'media/', 'profiles/',
                   'static/', 'templates/']
        for folder in folders:
            server.watch(folder)

        # Add watch: Compass & Sass
        compass_info = dict(
            compass='bundle exec compass',
            sass_dir='./compass/sass',
            css_dir='./static/site/css'
            )
        cmd = '{compass} compile --sass-dir {sass_dir} --css-dir {css_dir}'.format(
                **compass_info
                )
        server.watch('compass/sass/', shell(cmd=cmd, shell=True))

        # This is to make static files actually work in the development server.
        from django.contrib.staticfiles.urls import staticfiles_urlpatterns
        import codermoji.urls
        codermoji.urls.urlpatterns += staticfiles_urlpatterns()

        # Serve with livereload
        server.serve()
    else:
        execute_from_command_line(sys.argv)

```

As you can see, in the section for `livereload`, One of the parts executes compass on my `sass` files whenever they change, and livereload _injects_ the changes css into my browser window.